# bioimageio.core usage examples

In [ ]:
import os
import hashlib

import bioimageio.core
import imageio
# we use napari for visualising images, you can install it via `pip install napari` or`conda install napari`
import napari
import numpy as np
import xarray as xr

In [ ]:
# helper function for showing multiple images in napari
def show_images(*images, names=None):
    v = napari.Viewer()
    for i, im  in enumerate(images):
        name = None if names is None else names[i]
        if isinstance(im, str):
            im = imageio.imread(im)
        v.add_image(im, name=name)

## Loading a model

We will use a model that predicts foreground and boundaries in images of nuclei from the [kaggle nucles segmentation challenge](https://www.kaggle.com/c/data-science-bowl-2018).
Find the model on bioimage.io here: https://bioimage.io/#/?id=10.5072%2Fzenodo.881940

First, we will use `bioimageio.core.load_resource_description` to load the model and inspec the obtained model resource.

In [ ]:
# the model can be loaded using different representations:

# the doi of the zenodo entry corresponding to the model
rdf_doi = "10.5281/zenodo.6287342"

# the url of the yaml file containing the model resource description
rdf_url = "https://zenodo.org/record/6287342/files/rdf.yaml"

# filepath to the downloaded model (either zipped package or yaml)
# to download it from the website:
# - go to https://bioimage.io/#/?id=10.5281%2Fzenodo.5764892%2F5764893
# - click the download icon
# - select "ilastik" weight format
rdf_path = "/home/pape/Downloads/nuclei-segmentation-boundarymodel_pytorch_state_dict.zip"

In [ ]:
# load model from link to rdf.yaml
model_resource = bioimageio.core.load_resource_description(rdf_url)

In [ ]:
# load model from doi
model_resource = bioimageio.core.load_resource_description(rdf_doi)

In [ ]:
# load model from path to the zipped model files
model_resource = bioimageio.core.load_resource_description(rdf_path)

In [ ]:
# the "model_resource" instance returned by load_resource_description
# contains the information stored in the resource description (see https://github.com/bioimage-io/spec-bioimage-io/blob/gh-pages/model_spec_latest.md)

# we can e.g. check what weight formats are available in the model (pytorch_state_dict for the model used here)
print("Available weight formats for this model:", model_resource.weights.keys())
# or where the (downloaded) weight files are stored
print("Pytorch state dict weights are stored at:", model_resource.weights["pytorch_state_dict"].source)
print()
# or what inputs the model expects
print("The model requires as inputs:")
for inp in model_resource.inputs:
    print("Input with axes:", inp.axes, "and shape", inp.shape)
print()
# and what the model outputs are
print("The model returns the following outputs:")
for out in model_resource.outputs:
    print("Output with axes:", out.axes, "and shape", out.shape)

In [ ]:
# the function 'test_model' from 'bioimageio.core.resource_tests' can be used to fully test the model,
# including running prediction for the test input(s) and checking that they agree with the test output(s)
# before using a model, it is recommended to check that it properly works with this function
# 'test_model' returns a dict with 'status'='passed'/'failed' and more detailed information
from bioimageio.core.resource_tests import test_model
test_result = test_model(model_resource)
if test_result["status"] == "failed":
    print("model test:", test_result["name"])
    print("The model test failed with:", test_result["error"])
    print("with the traceback:")
    print("".join(test_result["traceback"]))
else:
    test_result["status"] == "passed"
    print("The model passed all tests")

## Prediction with the model

`bioimageio.core` implements functionality to run prediction with models in the `bioimage.io` format.
This includes functions to run prediction with `xarray.DataArrays` as input and convenience functions to run predictions for images stored on disc.

In [ ]:
# Load the example image for this model, which is stored in numpy file format.
input_image = np.load(model_resource.test_inputs[0])

In [ ]:
# Create an xarray.DataArray from the input image.
# DataArrays are like numpy arrays, but they have annotated axes.
# The axes are used to validate that the axes of the input image match the axes expected by a model.
input_array = xr.DataArray(input_image, dims=tuple(model_resource.inputs[0].axes))
# print the axis annotations ('dims') and the shape of the input array
print(input_array.dims)
print(input_array.shape)

In [ ]:
# Next, create a 'prediction_pipeline'. The prediction_pipeline is used to run prediction with a given model.
# This means it applies the preprocessing, runs inference with the model and applies the postprocessing.

# The 'devices' argument can be used to specify which device(s) to use for inference with the model.
# Hence it can be used to specify whether to use the cpu, a single gpu or multiple gpus (not implemented yet).
# By default (devices=None) a gpu will be used if available and otherwise the cpu will be used.
devices = None

# The 'weight_format' argument can be used to specify which weight format available in the model to use.
# By default (weight_format=None) the weight format with highest priority (as defined by bioimageio.core) will be used.
weight_format = None

prediction_pipeline = bioimageio.core.create_prediction_pipeline(
    model_resource, devices=devices, weight_format=weight_format
)

In [ ]:
# Use the prediction pipeline to run prediction for the image we loaded before.
# The prediction pipeline always returns a tuple (even if the model only has a single output tensor).
# So we access the first element of the prediction to get the predicted tensor.
prediction = prediction_pipeline(input_array)[0]
show_images(input_image, prediction, names=["image", "prediction"])  # show the prediction result

In [ ]:
# The prediction pipeline expects inputs to have a shape that fits the model exactly.
# So if the input does not fit the expected input shape the prediction will fail.
# E.g. if we crop the input to shape [1, 1, 250, 250] it will not work for our example model,
# which expects a spatial shape that is a multiple of 16
cropped_image = input_image[:, :, :250, :250]
cropped_array = xr.DataArray(cropped_image, dims=tuple(model_resource.inputs[0].axes))

In [ ]:
# Applying the prediction pipeline to an image with the wrong shape will fail!
prediction_pipeline(cropped_array)

In [ ]:
# Instead, we can use the function `predict_with_padding`, which will pad the image to a shape that fits the model.
prediction = bioimageio.core.predict_with_padding(prediction_pipeline, cropped_array)
show_images(cropped_image, prediction, names=["image", "prediction"])  # show the prediction result

In [ ]:
# There is also the function `predict_with_tiling`, which will run prediction for patches in a sliding window fashion.
# This is especially helpful for large inputs that do not fit into the model as a single input.

# The `tiling` argument is used to specify the tile size and the `halo`, which is the part of the patch
# that is cropped in order to reduce boundary artifacts.
# Alternatively, `tiling` can also be set to `True`, than the tile size and halo will be deduced from the model config
# (this is also the default behavior when the `tiling` parameter is not passed).
tiling = {"tile": {"x": 128, "y": 128}, "halo": {"x": 16, "y": 16}}  # use a tile size of 128x128 and crop a halo of 16 pixels

# if `verbose` is set to True a progress bar will be printed 
prediction = bioimageio.core.predict_with_tiling(prediction_pipeline, cropped_array, tiling=tiling, verbose=True)
show_images(cropped_image, prediction, names=["image", "prediction"]) 

### Convenience prediction functions

`bioimageio.core` also contains a few convenience functions to directly predict images that are stored on disc:
- `predict_image` can be used to run prediction for a single image
- `predict_images` to run prediction for many images

In [ ]:
# The convenience function `predict_image` can be used to run prediction for an image stored on disc.
from bioimageio.core.prediction import predict_image

# The filepath where the output should be stored; supports most common image formats as well as npy fileformat.
outputs = ["prediction.tif"]
predict_image(
    model_resource, model_resource.test_inputs, outputs
)

# The output tensor contains 2 channels, which is not supported by normal tif.
# Thus, these 2 channels are stored as 2 separate images.
fg_pred = imageio.imread("prediction-c0.tif")
bd_pred = imageio.imread("prediction-c1.tif")
show_images(input_image, fg_pred, bd_pred, names=["image", "foreground-prediction", "boundary-prediction"])

In [ ]:
# The convenience function `predict_images` can be use to run prediction for many images stored on disc
# Note: this only works for models which have a single input and output!
from bioimageio.core.prediction import predict_images

# Here we use a small subset of the dsb challenge data for prediction.
# The original data is available at https://github.com/stardist/stardist/releases/download/0.1.0/dsb2018.zip.
# We have added a few images to the repository so that the notebook runs out of the box.

# Get all paths to the images in the "example-images" folder.
from glob import glob
inputs = glob("./example-images/*.png")

# Create an output folder and specify the output path for each image.
output_folder = "./predictions"
os.makedirs(output_folder, exist_ok=True)
outputs = [os.path.join(output_folder, os.path.split(inp)[1]) for inp in inputs]

print(len(inputs), "images for prediction were found")

In [ ]:
# The model at hand can only predict images which have a spatial shape that is
# a multiple of 16. To run with images of other sizes we pass the `padding`
# argument to `predict_images` and specify that the input is padded to the next bigger
# size that is divisible by 16 (mode: dynamic).
# As an alternative `"mode": "fixed"` will pad to a fixed shape, e.g.
# `{"x": 512, "y": 512, "mode": "fixed"}` will always pad to a size of 512x512.
# The padding is cropped again after the prediction to restore the input shape.
padding = {"x": 16, "y": 16, "mode": "dynamic"}
predict_images(
    model_resource, inputs, outputs, padding=padding, verbose=True
)

# check the first input/output
show_images(inputs[0], outputs[0].replace(".png", "-c0.png"), outputs[0].replace(".png", "-c1.png"))

In [ ]:
# Instead of padding, we can also use tiling.
# Here, we specify a tile size of 256 and a halo (= what's cropped from the tile on either side) of 16.
tiling = {
    "tile": {"x": 256, "y": 256},
    "halo": {"x": 16, "y": 16},
}
predict_images(
    model_resource, inputs, outputs, tiling=tiling, verbose=True
)

# Check the first input output pair.
show_images(inputs[0], outputs[0].replace(".png", "-c0.png"), outputs[0].replace(".png", "-c1.png"))

## Create a biomiage.io model package

`bioimageio.core` also implements functionality to create a model package compatible with the [bioimageio model spec](https://github.com/bioimage-io/spec-bioimage-io/blob/gh-pages/model_spec_latest.md) ready to be shared via
the [bioimage.io model zoo](https://bioimage.io/#/).
Here, we will use this functionality to create two models, one that adds thresholding as post-processing to the outputs and another one that also adds weights in torchscript format.

In [ ]:
# get the python file defining the architecture.
# this is only required for models with pytorch_state_dict weights
def get_architecture_source(rdf):
    # here, we need the raw resource, which contains the information from the resource description
    # before evaluation, e.g. the file and name of the python file with the model architecture
    raw_resource = bioimageio.core.load_raw_resource_description(rdf)
    # the python file defining the architecture for the pytorch weihgts
    model_source = raw_resource.weights["pytorch_state_dict"].architecture
    # download the source file if necessary
    source_file = bioimageio.core.resource_io.utils.resolve_source(
        model_source.source_file
    )
    # if the source file path does not exist, try combining it with the root path of the model
    if not os.path.exists(source_file):
        source_file = os.path.join(raw_resource.root_path, os.path.split(source_file)[1])
    assert os.path.exists(source_file), source_file
    class_name = model_source.callable_name
    return f"{source_file}:{class_name}"

In [ ]:
# first new model: add thresholding of outputs as post-processing
# the convenience function `build_model` creates a biomageio model spec compatible package (=zipped folder)
from bioimageio.core.build_spec import build_model

# create a subfolder to store the files for the new model
model_root = "./new_model"
os.makedirs(model_root, exist_ok=True)

# create the expected output tensor (= outputs thresholded at 0.5)
threshold = 0.5
new_output = prediction > threshold
new_output_path = f"{model_root}/new_test_output.npy"
np.save(new_output_path, new_output)

# add thresholding as post-processing procedure to our model
preprocessing = [[{"name": prep.name, "kwargs": prep.kwargs} for prep in inp.preprocessing] for inp in model_resource.inputs]
postprocessing = [[{"name": "binarize", "kwargs": {"threshold": threshold}}]]

# get the model architecture
# note that this is only necessary for pytorch state dict models
model_source = get_architecture_source(rdf_doi)

# we use the `parent` field to indicate that the new model is created based on
# the nucleus segmentation model we have obtained from bioimage.io
# this field is optional and only needs to be given for models that are created based on other models from bioimage.io
# the parent is specified via it's doi and the hash of its rdf file
model_root_folder = os.path.split(model_resource.weights["pytorch_state_dict"].source)[0]
rdf_file = os.path.join(model_root_folder, "rdf.yaml")
with open(rdf_file, "rb") as f:
    rdf_hash = hashlib.sha256(f.read()).hexdigest()
parent = {"uri": rdf_doi, "sha256": rdf_hash}

# the name of the new model and where to save the zipped model package
name = "new-model1"
zip_path = os.path.join(model_root, f"{name}.zip")

# `build_model` needs some additional information about the model, like citation information
# all this additional information is passed as plain python types and will be converted into the bioimageio representation internally  
# for more informantion, check out the function signature
# https://github.com/bioimage-io/core-bioimage-io-python/blob/main/bioimageio/core/build_spec/build_model.py#L252
cite = [{"text": cite_entry.text, "url": cite_entry.url} for cite_entry in model_resource.cite]

# the training data used for the model can also be specified by linking to a dataset available on bioimage.io
training_data = {"id": "ilastik/stradist_dsb_training_data"}

# the axes descriptions for the inputs / outputs
input_axes = ["bcyx"]
output_axes = ["bcyx"]

# the pytorch_state_dict weight file
weight_file = model_resource.weights["pytorch_state_dict"].source

# the path to save the new model with torchscript weights
zip_path = f"{model_root}/new_model2.zip"

# build the model! it will be saved to 'zip_path'
new_model_raw = build_model(
    weight_uri=weight_file,
    test_inputs=model_resource.test_inputs,
    test_outputs=[new_output_path],
    input_axes=input_axes,
    output_axes=output_axes,
    output_path=zip_path,
    name=name,
    description="nucleus segmentation model with thresholding",
    authors=[{"name": "Jane Doe"}],
    license="CC-BY-4.0",
    documentation=model_resource.documentation,
    covers=[str(cover) for cover in model_resource.covers],
    tags=["nucleus-segmentation"],
    cite=cite,
    parent=parent,
    architecture=model_source,
    model_kwargs=model_resource.weights["pytorch_state_dict"].kwargs,
    preprocessing=preprocessing,
    postprocessing=postprocessing,
    training_data=training_data,
)

In [ ]:
# load the new model from the zipped package, run prediction and check the result
new_model = bioimageio.core.load_resource_description(zip_path)
prediction = predict_numpy(new_model, input_image)
show_images(input_image, prediction, names=["input", "binarized-prediction"])

## Add different weight format and package model with new weights

In [ ]:
# `convert_weigths_to_pytorch_script` creates torchscript weigths based on the weights loaded from pytorch_state_dict
from bioimageio.core.weight_converter.torch import convert_weights_to_torchscript
# `add_weights` adds new weights to the model specification
from bioimageio.core.build_spec import add_weights

# the path to save the newly created torchscript weights
weight_path = os.path.join(model_root, "weights.torchscript")
convert_weights_to_torchscript(new_model, weight_path)

# the path to save the new model with torchscript weights
zip_path = f"{model_root}/new_model2.zip"
new_model2_raw = add_weights(new_model_raw, weight_path, weight_type="torchscript", output_path=zip_path)

In [ ]:
# load the new model from the zipped package, run prediction and check the result
new_model = bioimageio.core.load_resource_description(zip_path)
prediction = predict_numpy(new_model, input_image, weight_format="torchscript")
show_images(input_image, prediction, names=["input", "binarized-prediction"])

In [ ]:
# models in the biomageio.core format can also directly be exported as zipped packages
# using `bioimageio.core.export_resource_package`
bioimageio.core.export_resource_package(new_model2_raw, output_path="another_model.zip")